# Homework 7

#### Instructions:
- Please read these instructions as well as the questions carefully.
- Your best resource is likely the official documentation of the datascience package: http://data8.org/datascience.
- Feel free to use Excel to play with the data.
- Please complete this notebook by filling in the cells provided. Before you begin, execute the following cell. Each time you start Jupyter Notebook, you will need to execute this cell.
- Please be sure not to re-assign variables throughout the notebook. For example, if you use `max_price` in your answer to one question, do not re-assign it later on.
- Please be sure not to re-define functions throughout the notebook. For example, if a question asks you to define `sharpe_ratio()`, the notebook that you submit should contain only one definition of that function.

In [1]:
# don't change this cell; just run it
from datascience import *
import numpy as np

## A. Sampling Basketball Players

This exercise uses salary data and game statistics for basketball players from the 2014-2015 NBA season. The data was collected from [Basketball-Reference](http://www.basketball-reference.com) and [Spotrac](http://www.spotrac.com).

Run the next cell to load the two datasets.

In [2]:
player_data = Table.read_table('https://www.dropbox.com/s/lwyvwsr0wne4m0t/player_data.csv?raw=1')
salary_data = Table.read_table('https://www.dropbox.com/s/efc74hgzkeaty43/salary_data.csv?raw=1')
player_data.show(3)
salary_data.show(3)

Name,Age,Team,Games,Rebounds,Assists,Steals,Blocks,Turnovers,Points
James Harden,25,HOU,81,459,565,154,60,321,2217
Chris Paul,29,LAC,82,376,838,156,15,190,1564
Stephen Curry,26,GSW,80,341,619,163,16,249,1900


PlayerName,Salary
Kobe Bryant,23500000
Amar'e Stoudemire,23410988
Joe Johnson,23180790


#### Question 1
We would like to relate players' game statistics to their salaries. Compute a table called `full_data` that includes one row for each player who is listed in both `player_data` and `salary_data`.  It should include all the columns from `player_data` and `salary_data`, except the `"PlayerName"` column.

In [17]:
full_data = player_data.join('Name', salary_data,'PlayerName')
full_data

Name,Age,Team,Games,Rebounds,Assists,Steals,Blocks,Turnovers,Points,Salary
A.J. Price,28,TOT,26,32,46,7,0,14,133,62552
Aaron Brooks,30,CHI,82,166,261,54,15,157,954,1145685
Aaron Gordon,19,ORL,47,169,33,21,22,38,243,3992040
Adreian Payne,23,TOT,32,162,30,19,9,44,213,1855320
Al Horford,28,ATL,76,544,244,68,98,100,1156,12000000
Al Jefferson,30,CHO,65,548,113,47,84,68,1082,13666667
Al-Farouq Aminu,24,DAL,74,342,59,70,62,55,412,1100602
Alan Anderson,32,BRK,74,204,83,56,5,60,545,1276061
Alec Burks,23,UTA,27,114,82,17,5,52,374,3034356
Alex Kirk,23,CLE,5,1,1,0,0,0,4,507336


Basketball team managers would like to hire players who perform well but don't command high salaries. From this perspective, a very crude measure of a player's *value* to their team is the number of points the player scored in a season for every **\$1000 of salary** (*Note*: the `Salary` column is in dollars, not thousands of dollars). For example, Al Horford scored 1156 points and has a salary of **\$12 million.** This is equivalent to 12,000 thousands of dollars, so his value is $\frac{1156}{12000}$.

#### Question 2
Create a table called `full_data_with_value` that's a copy of `full_data`, with an extra column called `"Value"` containing each player's value (according to our crude measure).

In [1]:
v = full_data.column('Points')/(full_data.column('Salary')/1000)
full_data_with_value = Table.copy(full_data).with_column('Value', v)
full_data_with_value.sort('Value').show()

NameError: name 'full_data' is not defined

Now suppose we weren't able to find out every player's salary (perhaps it was too costly to interview each player). Instead, we have gathered a *simple random sample* of 100 players' salaries. The cell below loads those data.

In [20]:
sample_salary_data = Table.read_table('https://www.dropbox.com/s/z1p315y1qa4on4j/sample_salary_data.csv?raw=1')
sample_salary_data.show(3)

PlayerName,Salary
C.J. Watson,2106720
Taj Gibson,8000000
Jerrelle Benimon,35000


#### Question 3
Entries in `sample_salary_data` constitute a sample. What do we call the entries in `salary_data`?

In [68]:
sample_vs_ = 'population'
sample_vs_

'population'

#### Question 4
The average salaries in `sample_salary_data` and `salary_data` are different because of sampling error. What is that difference?

In [29]:
sampling_error = np.mean(sample_salary_data.column('Salary')) - np.mean(salary_data.column('Salary'))
sampling_error

551546.7337398371

## B. Daily Market Returns

The next cell loads a table containing **all daily market returns** in the CRSP database (http://www.crsp.org) as of September 2020. Let's think of this as our **population**.

In [30]:
ff_daily = Table.read_table('https://www.dropbox.com/s/syxf1wvej9ktqwr/F-F_Research_Data_Factors_daily.CSV?raw=1')
ff_daily

Date,Mkt-RF,SMB,HML,RF
19260701,0.1,-0.24,-0.28,0.009
19260702,0.45,-0.32,-0.08,0.009
19260706,0.17,0.27,-0.35,0.009
19260707,0.09,-0.59,0.03,0.009
19260708,0.21,-0.36,0.15,0.009
19260709,-0.71,0.44,0.56,0.009
19260710,0.62,-0.5,-0.15,0.009
19260712,0.04,0.03,0.54,0.009
19260713,0.48,-0.26,-0.23,0.009
19260714,0.04,0.09,-0.48,0.009


We're interested in the distribution of daily market returns (in `"Mkt-RF"`), and generally, we won't have access to the population. Instead, if we sample correctly, we can use the data on a small number of days to get an idea about the distribution of returns in the population.

#### Question 1
In the following lines of code, we take two different samples from `ff_daily`. Decide which one of these samples is more representative of the population and calculate the sample mean using that sample.

In [41]:
sample1 = ff_daily.sort('Mkt-RF', descending=True).take(np.arange(100))
sample2 = ff_daily.take(np.arange(100))

a_sample_mean = np.mean(sample2.column('Mkt-RF'))
a_sample_mean

0.026799999999999997

#### Question 2
Write code to produce a random sample of size 500 from `ff_daily` and calculate the average daily market return in this sample. Assign these to `representative_sample` and `representative_mean` respectively.

*Hint:* In the demo for Lecture 11, we produce random samples in two different ways.

In [42]:
np.random.seed(12345) # DO NOT CHANGE THIS LINE OF CODE
representative_sample = ff_daily.take(np.random.choice(ff_daily.num_rows, 500))
representative_mean = np.average(representative_sample.column('Mkt-RF'))
representative_mean

0.031520000000000006

#### Question 3
Find the official documentation for NumPy on the Internet and describe the difference between the functions average() and mean() in the package. Assign `more_versatile` to either 1 or 2 depending on if you think average() or mean() is more versatile.

*Hint:* Recall from your introductory investments class that the return of a portfolio is a weighted average of the underlying asset returns.

In [43]:
more_versatile = 1
more_versatile

1

#### Question 4
Suppose we want to figure out what the highest daily market return is in our population, but we are tasked with doing this with only a sample of 500 from `ff_daily`.

To determine whether trying to find the highest daily market return from a sample is a plausible idea, write code that simulates the maximum of a random sample of size 500 from `ff_daily` 10000 times. Assign your array of maximums to `maximums`.

*Hint:* You may find it useful to revisit the notebook for Lecture 11.

In [45]:
maximums = make_array()

for i in np.arange(10000):
    sample = ff_daily.sample(500).column('Mkt-RF')
    new_maximum = max(sample)
    maximums = np.append(maximums, new_maximum)
maximums

array([ 3.97, 15.76, 11.35, ...,  3.91,  8.67,  6.32])

#### Question 5
We want to see if a random sample of size 500 is likely to help you determine the highest daily market return in the population. To help determine this, find the (actual) highest daily market return.

In [46]:
actual_highest = np.max(ff_daily.column('Mkt-RF'))
actual_highest

15.76

## Submission

Once you're finished, select "Save and Checkpoint" in the File menu. Pay attention to the name of your notebook---it's to the right of the "jupyter" logo at the upper left-hand corner of the page. On your computer your notebook has a file extension .ipynb. Locate and submit it on Gradescope.